In [1]:
%load_ext autoreload
%autoreload 2

import logging
from dotenv import load_dotenv
load_dotenv()

logging.basicConfig(level=logging.INFO)

from src.utils.notebook import set_project_root_for_notebook
set_project_root_for_notebook()

logger = logging.getLogger(__name__)

INFO:src.environment:Target: DEV
INFO:src.environment:Developer: FDDIFERD
INFO:src.environment:Schema Name: DEV_FDDIFERD
INFO:src.environment:Use Cache: False
INFO:src.utils.notebook:Current working directory: /Users/donatodiferdinando/Documents/GitHub/ml-snowflake-template


In [2]:
from projects.pltv import (
    Level,
    get_session,
    clean_df,
    DatasetLoader,
    ModelService,
)
session = get_session()

INFO:snowflake.snowpark:AST state has not been set explicitly. Defaulting to ast_enabled = True.
/Users/donatodiferdinando/Documents/GitHub/ml-snowflake-template/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ERROR:prophet.plot:Importing plotly failed. Interactive plots will not work.
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.18.0, Python Version: 3.10.18, Platform: macOS-26.2-arm64-arm-64bit
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain
INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 1.42.0,
"python.version" : 3.10.18,
"python.connector.version" : 3.18.0,
"python.connector.session.id" : 23301370166878,
"os.name" : Darwin

INFO:src.connection.session:Using database ML_LAYER_PLTV_DB and schema DEV_FDDIFERD
I

In [3]:
loader = DatasetLoader(session)
df = loader.load(Level.CAMPAIGN)
df.head()

INFO:projects.pltv.data.loader:Fetching CAMPAIGN dataset from Snowflake
INFO:snowflake.ml.feature_store.feature_store:Successfully connected to feature store: ML_LAYER_PLTV_DB.DEV_FDDIFERD.
INFO:projects.pltv.data.dataset:Spine SQL: 
with base as (
    select
        gross_add__created__day,
        brand,
        sku_type,
        channel,
        traffic_source,
        coalesce(campaign, 'NONE') as campaign,
        gross_add__type,
        plan__offer_type,
        plan__is_promo,
        plan__promo_price,
        plan__promo_days,
        plan__recurring_price,
        plan__recurring_days,
        gross_adds,
        gross_adds_canceled_day_one,
        gross_adds_canceled_day_three,
        gross_adds_canceled_day_seven
    from bi_layer_db.prod.exp_pltv_gross_add_metrics
    where gross_add__created__day >= '2021-01-01' and brand <> 'USSearchProfessional'
)

select
    date_trunc(month, gross_add__created__day) COHORT_MONTH,
    GROSS_ADD__TYPE, BRAND, SKU_TYPE, CHANNEL, TRAFF

,COHORT_MONTH,GROSS_ADD__TYPE,BRAND,SKU_TYPE,CHANNEL,TRAFFIC_SOURCE,CAMPAIGN,PLAN__IS_PROMO,GROSS_ADDS_CREATED_OVER_1_DAYS_AGO,GROSS_ADDS_CREATED_OVER_3_DAYS_AGO,...,NET_BILLINGS_30_DAYS,NET_BILLINGS_60_DAYS,NET_BILLINGS_90_DAYS,NET_BILLINGS_180_DAYS,NET_BILLINGS_365_DAYS,NET_BILLINGS_730_DAYS,AVG_CROSS_SELL_PRICE,CROSS_SELL_ADDS_ONE_DAY_SINCE_GROSS_ADD,CROSS_SELL_ADDS_THREE_DAYS_SINCE_GROSS_ADD,CROSS_SELL_ADDS_SEVEN_DAYS_SINCE_GROSS_ADD
0,2022-11-01,new,Intelius,ReversePhoneLookup,PaidSearch,5,bing_ads_638017806,True,2,2,...,0.950000,0.950000,0.950000,0.950000,0.950000,0.95,NaN,NaN,NaN,NaN
1,2024-09-01,new,InstantCheckmate,ReversePhoneLookup,MediaDisplay,871,adwords_21719135956,False,8,8,...,180.779999,218.759995,269.720001,287.690002,287.690002,NaN,14.096666,7.0,7.0,7.0
2,2024-01-01,new,TruthFinder,ReversePhoneLookup,PaidSearch,5,bing_ads_602166216,False,1,1,...,65.080002,103.110001,131.160004,169.190002,169.190002,NaN,28.049999,1.0,1.0,1.0
3,2024-10-01,new,Intelius,ReversePhoneLookup,PaidSearch,1413,adwords_18137370378,True,731,731,...,7760.529785,11725.059570,14559.839844,20245.320312,26075.460938,NaN,4.238562,279.0,280.0,281.0
4,2025-03-01,new,InstantCheckmate,Background,Affiliate,648,NONE,False,6818,6818,...,283673.125000,342356.031250,397803.625000,509761.375000,NaN,NaN,4.427945,3568.0,3594.0,3613.0


In [4]:
df[['GROSS_ADDS', 'BRAND', 'SKU_TYPE', 'AVG_CROSS_SELL_PRICE' ]].head(100)

,GROSS_ADDS,BRAND,SKU_TYPE,AVG_CROSS_SELL_PRICE
0,2,Intelius,ReversePhoneLookup,NaN
1,8,InstantCheckmate,ReversePhoneLookup,14.096666
2,1,TruthFinder,ReversePhoneLookup,28.049999
3,731,Intelius,ReversePhoneLookup,4.238562
4,6818,InstantCheckmate,Background,4.427945
...,...,...,...,...
95,13,OmniWatch,BasicMonthly,NaN
96,107,TruthFinder,Background,6.766956
97,10,Intelius,ReversePhoneLookup,14.925000
98,2,Intelius,Background,NaN


In [ ]:
import pandas as pd
fp = 'app/projects/pltv/data/cache/CAMPAIGN.parquet'
df = pd.read_parquet(fp)

In [ ]:
df.head()


In [ ]:
from projects import set_project_root_for_notebook
set_project_root_for_notebook()

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from projects.core.chargebacks_disputes_analysis.main import Service

svc = Service(use_cache=False)
date = (datetime.now() - relativedelta(months=6)).strftime("%Y-%m-%d")
svc.set_df(date)

In [ ]:
from projects.core.chargebacks_disputes_analysis.main import Level

level = [
    Level.PROCESSOR,
    Level.CARD_NETWORK, 
    # Level.BRAND
]

svc.get_ratio_df(level)